In [1]:
import os
import shutil
import pandas as pd
from pytube.exceptions import VideoUnavailable
from pytube import YouTube
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

In [2]:
test_path = '11-11-21_03-17-22_LEGACY_test.csv'
df = pd.read_csv(test_path)
df = df
df = df.reset_index()  # make sure indexes pair with number of rows
df.head()


,index,label,youtube_id,time_start,time_end,split,clarity_level,bad_egg
0,0,cr,c8j3h7fuYsY8,114,119,test,NaN,NaN
1,1,cr,MTJlyb8Odlc9,515,545,test,NaN,NaN
2,2,cr,UBNRj1eRWLs28,486,500,test,NaN,NaN
3,3,br,270F8s5TEKY17,644,654,test,NaN,NaN
4,4,st,ixc5jas6U3Q3,45,46,test,NaN,NaN


In [3]:
broken_videos = set()

def retry_download(tries, url, file_name, label):
    for i in range(tries):
            try:
                video = YouTube(url)
                yt_video = video.streams.get_highest_resolution()
                yt_video.download(output_path=f'videos/{label}/', filename=f"{file_name}.mp4")

                # download videos into correct folder
                ffmpeg_extract_subclip(filename=f'videos/{label}/{file_name}.mp4', t1=start, t2=end, targetname=f'videos/{label}/{file_name}_{start}_{end}.mp4') 
                os.remove(f'videos/{label}/{file_name}.mp4')
                
            except VideoUnavailable:
                if i < tries:
                    broken_videos.add(file_name)
                    continue

In [4]:
# Iterates all videos in csv
for index, row in df.iterrows():
    file_name = str(row['youtube_id'])
    start = int(row['time_start'])
    end = int(row['time_end'])
    label = str(row['label'])
    url = f'https://www.youtube.com/watch?v={file_name[0:11]}'
    fill_start = str(start).zfill(6)
    fill_end = str(end).zfill(6)

    if os.path.exists(f"videos/{label}/{file_name}_{start}_{end}.mp4"):
            print('file already exists')
            continue
    
    try:
        video = YouTube(url)
        yt_video = video.streams.get_highest_resolution()
        yt_video.download(output_path=f'videos/{label}/', filename=f"{file_name}.mp4")

        # download videos into correct folder
        
        ffmpeg_extract_subclip(filename=f'videos/{label}/{file_name}.mp4', t1=start, t2=end, 
                                targetname=f'videos/{label}/{file_name}_{fill_start}_{fill_end}.mp4') 
        os.remove(f'videos/{label}/{file_name}.mp4')

    except:
        print('retrying...', file_name)
        retry_download(1, url, file_name, label)      


Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
retrying... UBNRj1eRWLs28


In [ ]:
# Deletes broken videos
for video in broken_videos:
    df = df.drop(df.loc[df['youtube_id'] == video].index)

In [ ]:
# Creates a list with all labels from label_map.txt
with open('videos\label_map.txt', 'r') as f:
    label_list = [line.strip() for line in f]


In [ ]:
# Drops all rows where df['label'] != to labels in label_list
for label in df['label']:
    if label not in label_list:
        df = df.drop(df.loc[df['label'] == label].index)
    
    

In [ ]:
# Finding 35 percent of the least common label
least_label_35 = int(df['label'].value_counts().min()*.35)
# Groups randomized youtube_id's with corresponding labels, time_start, time_end
df_val = df.groupby('label')[['youtube_id', 'time_start', 'time_end']].apply(lambda s: s.sample(least_label_35))
    
    
    

In [ ]:
df_val = pd.DataFrame(df_val)
df_val = df_val.reset_index()
df_validation = df_val.drop(columns = 'level_1')

In [ ]:
# move val videos to new directory
for ind, row in df_val.iterrows():
    id = str(row['youtube_id'])
    label = str(row['label'])
    start = row['time_start']
    end = row['time_end']
    fill_start = str(start).zfill(6) # make times the same number of digits
    fill_end = str(end).zfill(6)

    if label not in label_list:
        continue
    
    newpath = f'val/{label}' 
    if not os.path.exists(newpath):
        os.makedirs(newpath)

    shutil.move(f"videos/{label}/{id}_{fill_start}_{fill_end}.mp4", f"val/{label}/{id}_{fill_start}_{fill_end}.mp4")

FileNotFoundError: [Errno 2] No such file or directory: 'videos/brawling/pUtcfItkY8025_000185_000190.mp4'